In [369]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [370]:
data = pd.read_csv("C:\Users\Niku\Documents\ML Project\data1.csv")
print data.shape
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['DateOnly'] = data['Date'].dt.day
print data.shape

(3400, 13)
(3400, 16)


In [435]:
#X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.15
#X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly']] #Acc = 0.16
#X = data[['DayOfWeek',  'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.155
X = data[[ 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.17 with test size 0.05
print X.shape
X = X.values
y = data['Category']
y = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state=1)
print X_train.shape
print X_test.shape

(3400, 7)
(3230L, 7L)
(170L, 7L)


In [436]:
def Prob(y_vector):
    total = len(y_vector)
    r_dict = {}
    cnt = Counter(y_vector)
    for key, value in cnt.items():
        r_dict[key] = float(value)/total
    return r_dict
    
dic = {}  
#ClassifyingByLabels 
def ClassifyByLabels(X_matrix, y_vector):
    
    [rows, columns] = np.shape(X_matrix)
    total = rows
    labels = np.unique(y_vector)
    for label in labels:
        dic[label] = defaultdict(list)
    for label in labels:
        row_indices = np.where(y_vector == label)[0] 
        X_current = X_train[row_indices, :]
        for i in range(0,columns):
            dic[label][i] += list(X_current[:,i])
    for label in labels:
        for col in range(0, columns):
            dic[label][col] = Prob(dic[label][col])
            #print dic[label][col]
            
ClassifyByLabels(X_train, y_train)   
crime_dict = Prob(y_train)

In [437]:
#Output maximum probability category
def maximum(dict1):
    values = dict1.values()
    keys = dict1.keys()
    max_index = values.index(max(values))
    max_key = keys[max_index]
    return max_key

#Testing on X_test

def Prediction(elem):
    predict_dict = {}
    for label in labels:
        prior = crime_dict[label]
        likelihood = 1.0
        for i in range(0, len(elem)):
            if elem[i] in dic[label][i].keys():
                likelihood *= dic[label][i][elem[i]]
            else:
                likelihood *= 0.0
        predict_dict[label] = likelihood
    return maximum(predict_dict)   
        

def classify(X_matrix):
        predicted_y = []
        no_rows, no_cols = np.shape(X_matrix)
        for i in range(0,no_rows):
            elem = X_matrix[i,:]
            prediction = Prediction(elem)
            predicted_y.append(prediction)
        return predicted_y

classify(X_test)
predicted_y = classify(X_test)
y_labels = np.unique(y_test)
tt = np.array(predicted_y)

for y_label in y_labels:
    f1 = f1_score(tt, y_test, y_label, average="weighted")
    #print("F1-score on the test-set for class %s is: %s" % (y_label, f1))

In [439]:
accuracy_score(y_test, tt)

0.17058823529411765

In [440]:
from sklearn.metrics import classification_report
print(classification_report(y_test, tt, target_names=labels))

                             precision    recall  f1-score   support

                      ARSON       0.00      0.00      0.00         1
                    ASSAULT       0.46      0.30      0.36        20
                    BRIBERY       0.00      0.00      0.00         1
                   BURGLARY       0.25      0.14      0.18         7
         DISORDERLY CONDUCT       0.00      0.00      0.00         2
DRIVING UNDER THE INFLUENCE       1.00      0.50      0.67         2
              DRUG/NARCOTIC       0.56      0.45      0.50        11
                DRUNKENNESS       0.00      0.00      0.00         1
               EMBEZZLEMENT       0.00      0.00      0.00         1
                  EXTORTION       1.00      0.50      0.67         2
     FORGERY/COUNTERFEITING       0.00      0.00      0.00         2
                      FRAUD       0.00      0.00      0.00         1
                 KIDNAPPING       0.22      0.11      0.15        18
              LARCENY/THEFT      